# 02_Database Integration (SQLite)
**Goal:** Establish a relational database to store the raw scraped data for structured querying and persistence.

**Steps Covered:**
1.  Initialize SQLite Database & Schema.
2.  Load raw data from CSV.
3.  Ingest data into the database.
4.  Verify data integrity and types.

### Import Libraries

In [1]:
import sqlite3
import pandas as pd
import os

print("imports are loaded")

imports are loaded


### Database Initialization
- Creating a connection to `car_data.db`.
- Defining the schema (tables and columns).

In [2]:
# --- CONFIGURATION ---
DB_NAME = "data/ebay_cars.db"
BASE_DIR = os.path.dirname(os.getcwd())
DB = os.path.join(BASE_DIR, DB_NAME)

# Create a connection
conn = sqlite3.connect(DB)
cursor = conn.cursor()

# 1. DROP the table if it exists
cursor.execute('DROP TABLE IF EXISTS listings')

# 2. CREATE the table with your custom Schema
create_table_query = '''
CREATE TABLE listings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    price REAL,
    year INTEGER,
    mileage INTEGER,
    brand TEXT,
    condition TEXT,
    bids REAL,
    seller_score TEXT,
    reviews_count INTEGER,
    link_type TEXT,
    status TEXT
)
'''
cursor.execute(create_table_query)
conn.commit()

conn.close()

print("Database and Table initialized successfully.")


Database and Table initialized successfully.


### Load Raw Data
- Reading the `ebay_cars_raw.csv` file into a Pandas DataFrame.

In [3]:
# CSV path to read data and store in DF
csv_file = "data/ebay.csv"
csv_path = os.path.join(BASE_DIR, csv_file)


# Check if it exists 
if os.path.exists(csv_path):
    # Load into a DF
    df = pd.read_csv(csv_path)
    
    print("Data loaded successfully!")
    display(df.head())
else:
    print("Error: Could not find the CSV file")

Data loaded successfully!


,title,price,year,mileage,brand,condition,bids,seller_score,reviews_count,link_type,status
0,2004 BMW 5-Series 525i,1750.0,2004,171100,BMW,Pre-Owned,45.0,99%,10300,Auction,Active
1,2007 Honda Accord,660.0,2007,256860,Honda,Pre-Owned,26.0,99%,10300,Auction,Active
2,2019 Nissan Altima 2.5 Platinum AWD clean carf...,5100.0,2019,16550,Nissan,Great Value Look,60.0,100%,2300,Buy It Now,Active
3,New Listing2016 Q50 3.0T PREMIUM NAV BK/CAM RO...,11995.0,2016,93981,Infiniti,Pre-Owned,NaN,100%,888,NaN,Active
4,2013 Toyota Avalon XLE Premium 4dr Sedan,7700.0,2013,57449,Toyota,AmeriCarsUSA Hollywood 305-331-5208,25.0,96%,1500,Buy It Now,Active


### Data Ingestion (ETL)
- Writing the DataFrame into the SQLite table `cars`.
- Using `if_exists='replace'` to ensure a fresh start.

In [5]:
# Save to DB
conn = sqlite3.connect(DB)
    
# 'if_exists="append"
df.to_sql("listings", conn, if_exists="append", index=False)
    
conn.close()
print(f"Saved {len(df)} rows to '{DB_NAME}'")

Saved 4628 rows to 'data/ebay_cars.db'


### Schema & Type Verification
- Confirming that Pandas object types correctly mapped to SQLite types (TEXT, INTEGER, REAL).

In [6]:
# Display data types of each column
print("Data Types:")
print(df.dtypes)

print("\nData Summary:")
print(f"  Numeric columns: {df.select_dtypes(include=['int64', 'float64']).columns.tolist()}")
print(f"  Text columns: {df.select_dtypes(include=['object']).columns.tolist()}")

Data Types:
title             object
price            float64
year               int64
mileage            int64
brand             object
condition         object
bids             float64
seller_score      object
reviews_count      int64
link_type         object
status            object
dtype: object

Data Summary:
  Numeric columns: ['price', 'year', 'mileage', 'bids', 'reviews_count']
  Text columns: ['title', 'brand', 'condition', 'seller_score', 'link_type', 'status']


# Data Summary

In [7]:
# Display statistical summary of numeric columns
print("Statistical summary:")
df.describe()

Statistical summary:


,price,year,mileage,bids,reviews_count
count,4628.000000,4628.000000,4.628000e+03,228.000000,4628.000000
mean,23131.871493,2009.363224,1.034482e+05,5.552632,1003.316768
std,31829.138176,20.009067,1.815358e+06,10.414825,5597.048744
min,1.250000,1908.000000,0.000000e+00,0.000000,0.000000
25%,9250.000000,2010.000000,3.737175e+04,0.000000,0.000000
50%,15844.000000,2016.000000,6.993150e+04,0.000000,230.000000
75%,25000.000000,2020.000000,1.046615e+05,6.000000,771.000000
max,849980.000000,2026.000000,1.234568e+08,60.000000,158200.000000


# Phase 2: Database Integration Summary

### 1. The Goal: "Data Persistence"
Raw CSV files are good for transport, but bad for querying. To enable complex analysis later (like "Find all BMWs under $10k"), I moved the data into a relational database.
* **Source:** `ebay.csv` (Flat text file).
* **Destination:** `ebay_cars.db` (Structured SQLite Database).

### 2. The Tech Stack
* **SQLite3:** Chosen for its serverless architecture and ease of integration with Python. It allows for full SQL capabilities without the overhead of setting up a PostgreSQL/MySQL server.
* **Pandas:** Used as the ORM (Object Relational Mapper) layer to seamlessly transfer dataframes into SQL tables.

### 3. Key Achievements
* **Schema Definition:** Created a robust schema to hold heterogeneous data (Mixing strings for Titles, Integers for Year, and Floats for Price).
* **Automated Ingestion:** Built a reusable pipeline that reads the raw scrape output and commits it to the database automatically.
* **Data Integrity:** Verified that 100% of the CSV rows were successfully transferred and that data types (Numeric vs. Text) were preserved.

###  Next Step: Data Cleaning
Now that the data is safely stored in a database, the next phase is **Data Cleaning & Feature Engineering**.